# Exploratory Data Analysis

Hypothesis: We can use the https://arxiv.org/pdf/2406.04313 method for increasing honesty

In [1]:
# autoreload your package
%load_ext autoreload
%autoreload 2
import adapter_overseer


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
import warnings
# warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", ".*does not have many workers.*")
# warnings.filterwarnings("ignore", ".*divide by zero.*")

## numeric, plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (7.0, 4)

## utils
from pathlib import Path
from tqdm.auto import tqdm
import logging, os, re
import collections, functools, itertools
from loguru import logger

from typing import List, Callable, Tuple, Dict, Optional
from jaxtyping import Float, Int
from torch import Tensor

# torch
# import pytorch_lightning as pl
from einops import rearrange, repeat, reduce
import torch
import torch.nn as nn


from baukit.nethook import get_module
from baukit import TraceDict

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from adapter_overseer.config import ExtractConfig

cfg = ExtractConfig()
cfg

ExtractConfig(datasets=('amazon_polarity',), datasets_ood='imdb', model='failspy/Llama-3-8B-Instruct-abliterated', collection_layers=('base_model.model.model.layers.10', 'base_model.model.model.layers.20'), batch_size=2, prompt_format=None, num_shots=2, max_length=776, max_examples=1000, seed=42, max_epochs=1)

## Load

In [5]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# https://huggingface.co/blog/mlabonne/orpo-llama-3
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
torch_dtype, device

(torch.bfloat16, device(type='cuda', index=0))

In [7]:
# load model
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(load_in_4bit=True,     bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch_dtype, bnb_4bit_use_double_quant=True,)
model = AutoModelForCausalLM.from_pretrained(cfg.model, device_map="auto", quantization_config=quantization_config,)
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.60s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [8]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'
tokenizer.truncation_side = 'left'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# https://old.reddit.com/r/LocalLLaMA/comments/1coizjy/tokenizer_config_of_llama3_changed_by_meta_in_hf/
tokenizer.eos_token # it's good

'<|eot_id|>'

In [10]:
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# \peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
# )
# model = get_peft_model(model, peft_config)


In [11]:
# from peft import prepare_model_for_int8_training
# # we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in float32 for stability. We also cast the output of the last layer in float32 for the same reasons.
# model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

In [12]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model
# https://github.com/huggingface/peft/blob/main/src/peft/utils/constants.py
config = LoraConfig(
                        #r=32, lora_alpha=64, 
                    # target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none"
                    )


# LoRA config
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
# )
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


## Get data

In [13]:
# perhaps use load_preproc_datasets from sdb_probes_are_lie_detectors repo... /media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/src/prompts/prompt_loading.py

In [14]:
# load a dataset of paired prompts, to try and get the model to lie
from adapter_overseer.prompts.prompt_loading import load_preproc_datasets

N = cfg.max_examples
ds_tokens = load_preproc_datasets(
    cfg.datasets,
    tokenizer,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    max_length=cfg.max_length,
    prompt_format=cfg.prompt_format,
)
ds_tokens


2024-06-10 06:44:52.433 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:392 - median token length: 375.0 for amazon_polarity. max_length=776
2024-06-10 06:44:52.435 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:396 - truncation rate: 0.00% on amazon_polarity
/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:2328: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(
2024-06-10 06:44:52.704 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:405 - num_rows (after filtering out truncated rows) 3004=>3004


Dataset({
    features: ['ds_string', 'example_i', 'answer', 'messages', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'question', 'input_ids', 'attention_mask', 'truncated', 'length', 'prompt_truncated', 'choice_ids'],
    num_rows: 1001
})

## Train: transformers

https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb

In [21]:
# TODO change the loss function!
# we need to modify the forward pass, so that it returns a different loss function
# but to calculate this we will need to residuals now, and as they werre
# loss_bad = mse(repr_current, repr_target)

# from transformers import SFTTrainer
from trl.trainer import SFTTrainer, SFTConfig
import torch.nn.functional as F

from adapter_overseer.helpers.torch_helpers import clear_mem, switch
from adapter_overseer.helpers.scores import select_choices

class CustomSFTTrainer(SFTTrainer):
    """
    Custom SFTTrainer that orthoganalizes the repr of bad examples, and retains good repr of examples

    See: https://arxiv.org/pdf/2406.04313

    args:
        collection_layers: list of baukit layer names to collect
    """
    def __init__(self, *args, collection_layers: list, alpha=0.1, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.collection_layers = collection_layers
        self.alpha = alpha
        self.total_steps = self.args.max_steps

    def compute_loss(self, model, inputs, return_outputs=False):       

        batch = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}

        # collect the residuals of the model
        with model.disable_adapter():
            orig_outputs = model(**batch, output_hidden_states=True)
        outputs = model(**batch, output_hidden_states=True)

        def collect_hs(hidden_states):
            """The residual stream is the diff of the hs."""
            hs = [hidden_states[i] for i in self.collection_layers]
            return rearrange(hs, 'l b t h -> b l t h').diff(1)

        rep_adapt = collect_hs(outputs.hidden_states)
        rep_orig = collect_hs(orig_outputs.hidden_states)

        # so now we have a mixed batch of good and bad outputs
        # get probs of each choice
        # compare to labels to seperate into good and bad
        choice_ids = inputs['choice_ids'].detach().cpu().long()
        # label_instructed = inputs['label_true'] ^ inputs['instructed_to_lie']
        label_true = inputs['label_true']

        # does the underlying model get it right or wrong?
        end_logits = orig_outputs["logits"][:, -1]
        probs = torch.softmax(end_logits, -1)
        choice_probs = select_choices(probs, choice_ids).sum(2)
        binary_ans = choice_probs[:, 1] / (choice_probs.sum(1) + 1e-12)
        correct_truth_telling = switch(binary_ans, label_true)
        # correct_instruction_following = switch(binary_ans, label_instructed)

        mask_desired = correct_truth_telling>0.5


        # get coeffecient
        steps = self.state.global_step + 1
        c = torch.tensor(self.alpha * steps / (2 * self.total_steps)).to(rep_orig.dtype)
        loss_retain = F.mse_loss(rep_orig, rep_adapt, reduction='none' )[mask_desired]
        if loss_retain.numel() == 0:
            loss_retain = 0
        else:
            loss_retain = loss_retain.mean()
        loss_rr = F.relu(F.cosine_similarity(rep_orig, rep_adapt, dim=1))[~mask_desired]
        if loss_rr.numel() == 0:
            loss_rr = 0
        else:
            loss_rr = loss_rr.mean()
        loss = loss_rr * (1 - c) + c * loss_retain
        loss = loss
        logger.debug(f"steps: {steps}, c: {c}, loss_rr: {loss_rr:2.3f}, loss_retain: {loss_retain:2.3f}, loss={loss:2.3f}, mask_desired: {(mask_desired*1.0).mean():2.3f}")
        
        return (loss, outputs) if return_outputs else loss
    

# TODO make sure that multiple cols get passed into trainer
ds = ds_tokens.select_columns(['label_true', 'label_instructed' ,'instructed_to_lie', 'input_ids', 'attention_mask', 'choice_ids'])

import transformers

# see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L58
trainer = CustomSFTTrainer(
    model=model,
    train_dataset=ds,
    collection_layers=[10, 20],
    # max_seq_length=cfg.max_length,
    args=SFTConfig(
        # see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_config.py#L21
        max_seq_length=cfg.max_length,
        per_device_train_batch_size=4, # 18GB/24GB
        gradient_accumulation_steps=6, # we want to accumulate the gradients to make the batch size larger, so we have sufficient examples of good and bad behaviour to learn from
        warmup_steps=10,
        max_steps=2000,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        remove_unused_columns=False,
    ),
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
max_steps is given, it will override any value given in num_train_epochs
  0%|          | 0/2000 [00:00<?, ?it/s]/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
2024-06-10 06:54:27.903 | INFO     | __main__:compute_loss:77 - steps: 1, c: 2.499999936844688e-05, loss_rr: 0.988, loss_retain: 0.000, loss=0.988, mask_desired: 0.500
2024-06-10 06:54:33.321 | INFO     | __main__:compute_loss:77 - steps: 1, c: 2.4

{'loss': 0.9884, 'grad_norm': 0.06136675924062729, 'learning_rate': 2.9999999999999997e-05, 'epoch': 0.02}


2024-06-10 06:55:00.826 | INFO     | __main__:compute_loss:77 - steps: 2, c: 4.999999873689376e-05, loss_rr: 0.988, loss_retain: 0.000, loss=0.988, mask_desired: 0.500
2024-06-10 06:55:06.371 | INFO     | __main__:compute_loss:77 - steps: 2, c: 4.999999873689376e-05, loss_rr: 0.986, loss_retain: 0.000, loss=0.986, mask_desired: 0.500
2024-06-10 06:55:11.943 | INFO     | __main__:compute_loss:77 - steps: 2, c: 4.999999873689376e-05, loss_rr: 0.986, loss_retain: 0.000, loss=0.986, mask_desired: 0.500
2024-06-10 06:55:17.520 | INFO     | __main__:compute_loss:77 - steps: 2, c: 4.999999873689376e-05, loss_rr: 0.989, loss_retain: 0.000, loss=0.989, mask_desired: 0.500
2024-06-10 06:55:23.125 | INFO     | __main__:compute_loss:77 - steps: 2, c: 4.999999873689376e-05, loss_rr: 0.985, loss_retain: 0.000, loss=0.985, mask_desired: 0.000
2024-06-10 06:55:28.750 | INFO     | __main__:compute_loss:77 - steps: 2, c: 4.999999873689376e-05, loss_rr: 0.990, loss_retain: 0.000, loss=0.990, mask_desired

{'loss': 0.9872, 'grad_norm': 0.06063719466328621, 'learning_rate': 5.9999999999999995e-05, 'epoch': 0.05}


2024-06-10 06:55:34.393 | INFO     | __main__:compute_loss:77 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.988, loss_retain: 0.000, loss=0.987, mask_desired: 0.500
2024-06-10 06:55:40.011 | INFO     | __main__:compute_loss:77 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.988, loss_retain: 0.000, loss=0.988, mask_desired: 0.750
2024-06-10 06:55:45.631 | INFO     | __main__:compute_loss:77 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.000, loss_retain: 0.000, loss=0.000, mask_desired: 1.000
2024-06-10 06:55:51.217 | INFO     | __main__:compute_loss:77 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.978, loss_retain: 0.000, loss=0.978, mask_desired: 0.750
2024-06-10 06:55:56.851 | INFO     | __main__:compute_loss:77 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.984, loss_retain: 0.000, loss=0.984, mask_desired: 0.500
2024-06-10 06:56:02.484 | INFO     | __main__:compute_loss:77 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.984, loss_retain: 0.000, loss=0.984, mask_desired

...

{'loss': 0.2942, 'grad_norm': 5.933142185211182, 'learning_rate': 0.00029668341708542713, 'epoch': 0.76}


2024-06-10 07:12:47.326 | INFO     | __main__:compute_loss:77 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.283, loss_retain: 0.000, loss=0.283, mask_desired: 0.000
2024-06-10 07:12:53.065 | INFO     | __main__:compute_loss:77 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.287, loss_retain: 0.061, loss=0.287, mask_desired: 0.250
2024-06-10 07:12:58.820 | INFO     | __main__:compute_loss:77 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.296, loss_retain: 0.062, loss=0.295, mask_desired: 0.250
2024-06-10 07:13:04.601 | INFO     | __main__:compute_loss:77 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.291, loss_retain: 0.062, loss=0.291, mask_desired: 0.750
2024-06-10 07:13:10.368 | INFO     | __main__:compute_loss:77 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.290, loss_retain: 0.062, loss=0.289, mask_desired: 0.250
2024-06-10 07:13:16.134 | INFO     | __main__:compute_loss:77 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.306, loss_retain: 0.000, loss=0.306, mask_d

{'loss': 0.2917, 'grad_norm': 8.013928413391113, 'learning_rate': 0.0002965326633165829, 'epoch': 0.79}


2024-06-10 07:13:21.942 | INFO     | __main__:compute_loss:77 - steps: 34, c: 0.0008500000112690032, loss_rr: 0.283, loss_retain: 0.061, loss=0.283, mask_desired: 0.500
2024-06-10 07:13:27.703 | INFO     | __main__:compute_loss:77 - steps: 34, c: 0.0008500000112690032, loss_rr: 0.281, loss_retain: 0.060, loss=0.281, mask_desired: 0.500
2024-06-10 07:13:33.446 | INFO     | __main__:compute_loss:77 - steps: 34, c: 0.0008500000112690032, loss_rr: 0.280, loss_retain: 0.059, loss=0.280, mask_desired: 0.500
2024-06-10 07:13:39.215 | INFO     | __main__:compute_loss:77 - steps: 34, c: 0.0008500000112690032, loss_rr: 0.274, loss_retain: 0.061, loss=0.274, mask_desired: 0.750
2024-06-10 07:13:44.976 | INFO     | __main__:compute_loss:77 - steps: 34, c: 0.0008500000112690032, loss_rr: 0.297, loss_retain: 0.062, loss=0.296, mask_desired: 0.250


KeyboardInterrupt: 

In [ ]:
# save
model.save_pretrained("outputs")

# Eval

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import pipeline

accuracy = evaluate.load("accuracy")

input_texts = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")["text"]
input_texts = [s for s in input_texts[:200] if s!='']

def preprocess_function(examples):

    return tokenizer(examples["text"], truncation=True)

# https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/pipelines#pipeline-batching
batch_size = 4
pipe = pipeline("text-classification", device="auto", model=model, tokenizer=tokenizer)
    for out in tqdm(pipe(dataset, batch_size=batch_size), total=len(dataset)):
        pass